# リアルタイム推論サービスを作成する

予測モデルのトレーニング後、クライアントが新しいデータから予測を取得するために使用できるリアルタイム サービスとしてモデルをデプロイできます。

## ワークスペースに接続する

作業を開始するには、ワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## モデルをトレーニングして登録する

それでは、モデルをトレーニングして登録しましょう。

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="mslearn-train-diabetes")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## モデルを Web サービスとして公開する

糖尿病の可能性に基づいて患者を分類する機械学習モデルをトレーニングし、登録しました。このモデルは、糖尿病の臨床検査を受ける必要があるとリスクがあると考えられる患者のみが必要な医師の手術などの運用環境で使用できます。このシナリオをサポートするには、モデルを Web サービスとしてデプロイします。

まず、ワークスペースに登録したモデルを決定しましょう。

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

それでは、デプロイしたいモデルを取得しましょう。既定では、モデル名を指定すると、最新バージョンが返されます。

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

このモデルをホストする Web サービスを作成しますが、これにはコードと構成ファイルが必要です。そのため、それらのフォルダーを作成してみましょう。

In [ ]:
import os

# Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

モデルをデプロイする Web サービスでは、入力データを読み込み、ワークスペースからモデルを取得し、予測を生成して返すために、Python コードが必要になります。このコードは、Web サービスにデプロイされる*エントリ スクリプト* (頻繁に*スコアリング スクリプト*と呼ばれます) に保存します。

スクリプトは 2 つの関数で構成されています。

- **init**: この関数は、サービスが初期化されるときに呼び出され、通常、モデルをロードするために使用されます。スコアリング スクリプトは、**AZUREML_MODEL_DIR** 環境変数を使用して、モデルが保存されているフォルダーを決定することに注意してください。
- **run**: この関数は、クライアント アプリケーションが新しいデータを送信するたびに呼び出され、通常、モデルから予測を推測するために使用されます。

In [ ]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Web サービスはコンテナーでホストされ、コンテナーは初期化されるときに必要な Python 依存関係をインストールする必要があります。この場合、スコアリング コードには、スコアリング Web サービスで使用される **scikit-learn** といくつかの Azure Machine Learning 固有のパッケージが必要なので、これらを含む環境を作成します。次に、その環境をスコアリング スクリプトとともに*推論構成*に追加し、環境とスクリプトがホストされるコンテナーの*デプロイ構成*を定義します。

次に、これらの構成に基づいてモデルをサービスとしてデプロイできます。

> **詳細情報**: モデル デプロイ、ターゲット実行環境のオプションの詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)を参照してください。

デプロイは、最初にコンテナー イメージを作成するプロセスを実行し、そのイメージに基づいて Web サービスを作成するプロセスを実行するため、時間がかかります。デプロイが正常に完了すると、**正常**な状態が表示されます。

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment(name='service-env')
python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']
for package in python_packages:
    service_env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

うまくいけば、デプロイが成功し、**正常**な状態を確認できます。確認できない場合は、次のコードを使用して、トラブルシューティングに役立つサービス ログを取得できます。

In [ ]:
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

[Azure Machine Learning Studio](https://ml.azure.com) でワークスペースを確認し、ワークスペースにデプロイされたサービスを示す**エンドポイント**ページを表示します。

次のコードを実行して、ワークスペース内の Web サービスの名前を取得することもできます。

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## Web サービスを使用する

サービスをデプロイしたら、クライアント アプリケーションからサービスを使用できます。

In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

また、複数の患者の観察をサービスに送信し、それぞれの予測を取得することもできます。

In [ ]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

上記のコードでは、Azure Machine Learning SDK を使用してコンテナー化された Web サービスに接続し、それを使用して糖尿病分類モデルから予測を生成しています。運用環境では、Azure Machine Learning SDK を使用せず、単に Web サービスに HTTP 要求を行うビジネス アプリケーションによってモデルが使用される可能性があります。

これらのアプリケーションが要求を送信する必要がある URL を決定しましょう。

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

エンドポイント URI がわかったので、アプリケーションは HTTP 要求を行い、患者データを JSON 形式で送信し、予測されたクラスを受け取ることができます。

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

認証を必要としない Azure Container Instances (ACI) サービスとして Web サービスをデプロイしました。これは開発とテストには適していますが、運用環境では Azure Kubernetes Service (AKS) クラスターへのデプロイとトークンベースの認証の有効化を検討する必要があります。これには、**Authorization** ヘッダーを含める REST 要求が必要です。

## サービスを削除する

サービスが不要になった場合は、不要な料金が発生しないように削除する必要があります。

In [ ]:
service.delete()
print ('Service deleted.')

モデルをサービスとして公開する方法の詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)を参照してください。